In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import gradio as gr
import os

# Set the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 64
num_epochs = 100
latent_dim = 100
image_size = 64
output_dir = "generated_images"
model_save_path = "generator.pth"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define the Generator model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, image_size * image_size * 3),
            nn.Tanh()
        )

    def forward(self, z):
        generated_image = self.model(z)
        generated_image = generated_image.view(generated_image.size(0), 3, image_size, image_size)
        return generated_image

# Define the Discriminator model (same as before)
# ...

# Load the pre-trained generator if available
generator = Generator()
if os.path.exists(model_save_path):
    generator.load_state_dict(torch.load(model_save_path, map_location=device))
    generator.eval()
    print("Pre-trained generator loaded.")

# Define Gradio interface functions
def generate_output(input_image):
    input_tensor = preprocess_input(input_image)
    with torch.no_grad():
        latent_vector = torch.randn(1, latent_dim, device=device)
        generated_image = generator(latent_vector)
    output_image = postprocess_output(generated_image)
    return output_image

def preprocess_input(input_image):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    input_tensor = transform(input_image).unsqueeze(0).to(device)
    return input_tensor

def postprocess_output(output_tensor):
    denormalize = transforms.Normalize((-1, -1, -1), (2, 2, 2))
    output_tensor = denormalize(output_tensor)[0].permute(1, 2, 0)
    output_image = transforms.ToPILImage()(output_tensor.cpu())
    return output_image

# Gradio Interface
iface = gr.Interface(
    fn=generate_output,
    inputs="image",
    outputs="image",
    title="GAN Image Generator",
    description="Upload an image and see the GAN-generated output!",
    live=True
)

iface.launch()


c:\Users\samue\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\samue\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\routes.py", line 442, in run_predict
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\samue\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1392, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\samue\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1097, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\samue\AppData\Local\Programs\Python\Python311\Lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\samue\AppData\Local\Programs\Python\Python311\Lib\site